In [2]:
#importing the libraries
import pandas as pd
import json
import numpy as np
import seaborn as sns
from ast import literal_eval
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

pd.set_option('display.max_rows', None)

#reaidng the data set
df = pd.read_csv('movies_metadata.csv')

print("original_columns",df.columns)

#using the following columns for linear regression. Here Popularity acts as the label

columns = ['budget','revenue','runtime','vote_average','original_language','popularity']



columns1 = ['budget','revenue','runtime','vote_average']


df=pd.DataFrame(df,columns=columns)


#converting the data types to numeric types. There were many values of object type and was not compatible. So we had to convert all those to a suitable type

def convert_numeric(df,columns):
        #for each column passed convert it into numeric type
        for column in columns:
            df[column] = df[column].apply(pd.to_numeric,errors = 'coerce')
        return df

#normlsing the features so that gradient descent takes place in an appropriate manner and global optimum is obtained

def normalize(data):
        
        #using features scaling technnique(values - mean/ sigma)
        for column in columns1:
            mean=data[column].mean()
            std=data[column].std()
            data[column]=(data[column]-mean)/(std)
        return data
            
#data processing function it returns the null values

def data_preprocessing(df,column_name):
    df1 = df
    df1.loc[df1[column_name] ==0 , column_name] = np.NaN
    return df1

#There is a categorical feature called as original language that we have used. So converting it using one hot encoding. We are assuming each language gets equal importance. So we are not going with label encoding
def one_hot_encoding(data,column):
    data=pd.concat([data,pd.get_dummies(data[column],prefix=column)],axis=1)
    del data[column]
    return data
    

#replacing all the values conatining 0 (data not available) with np.NaN for easier data processing 
df=df.replace(0,np.NaN)

#dropping the null values

df=df.dropna()

#calling the numeric function

df = convert_numeric(df,columns1)



df = convert_numeric(df,['popularity'])

#further preprocessing

df.loc[df['budget'] <= 100 , ['budget']] = df['budget'].median()

df.loc[df['revenue'] <= 10000, ['revenue']] = df['revenue'].median()


df = df[df['popularity'] <= 30]

df = df[df['runtime']>=70]

df = df[df['runtime'] <=180 ]

print(df.describe())

#performing one hot encoding for original language

df = one_hot_encoding(df,column = 'original_language')

#numeric columns
df1 = df
df1 = pd.DataFrame(df1,columns = ['budget','revenue','runtime','vote_average'])

#normalising the features
df1=normalize(df1)

#extractig  the y lables
y=df['popularity']


#performing linear regression using K-Fold validation Technique

regr = LinearRegression()  

#performing 5 K Fold Validation

K_Fold_Validation = KFold(n_splits = 5,shuffle = True)

i = 0

for i in range(5):
    
    split = next(K_Fold_Validation.split(df1),None)
    
    
    X_train,X_test,y_train,y_test = df1.iloc[split[0]],df1.iloc[split[1]],y.iloc[split[0]],y.iloc[split[1]]
    
    
    #fitting the train set and labels 
    regr.fit(X_train, y_train) 
    
    #predicting the lables for X_test
    y_pred=regr.predict(X_test)
    
    
    
    #evaluation metrics by printing the mean absolute, squared and root mean sqaured errors.

    print("K -FOLD ", i)
    
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    
    print('Root Mean Squared Error on test data is:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    
    i = i + 1
    




/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


original_columns Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
             budget       revenue      runtime  vote_average   popularity
count  7.160000e+03  7.160000e+03  7160.000000   7160.000000  7160.000000
mean   2.405932e+07  6.373855e+07   107.254888      6.235642     7.752265
std    3.374732e+07  1.251818e+08    18.309540      0.917120     4.967478
min    1.130000e+02  1.001800e+04    70.000000      0.500000     0.000001
25%    8.000000e+06  3.355740e+06    94.000000      5.700000     4.009073
50%    8.000000e+06  1.707133e+07   104.000000      6.300000     7.417866
75%    2.800000e+07  6.451350e+07   117.000000      6.900000  